In [1]:
import time
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain,LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain
from dotenv import load_dotenv
load_dotenv()


True

In [13]:
persist_directory = "../dboffersv2"
embeddings = OpenAIEmbeddings()
offerdb = Chroma(
    persist_directory=persist_directory,
    embedding_function = embeddings
)

Using embedded DuckDB with persistence: data will be stored in: ../dboffersv2


In [21]:
from langchain.prompts.prompt import PromptTemplate
template = """"Tickit is a rewards programme brought to you by Dubai Holding. The document below contains list of brands or outlets enrolled in Tickit where users can earn and redeem points. 
Reply as bullet points when possible.
If you can't find the answer in the document, say you are unable to find the answer and propose him to reach out to the human agent.

Document: 
{context}\n\n

Current conversation:
{chat_history}

Question:
{question}\n\n

Answer:
"""
 
offer_prompt= PromptTemplate.from_template(template)

In [22]:
question = "food places in Dubai Hills"
similar_docs = offerdb.similarity_search(question, k=5)
similar_docs

[Document(page_content="NAME: Vietnamese Foodies Dubai Hills Mall\nLOCATION: Dubai Hills Mall\nDESCRIPTION: Vietnamese Foodies Dubai Hills Mall: Family restaurant. Vietnamese Foodies Dubai Hills Mall Address: Dubai Hills Mall - Ground floor - Entrance 1 - Dubai Hills - Dubai - United Arab Emirates. Vietnamese Foodies Dubai Hills Mall Hours: Closed ⋅ Opens 10\u202fAM. Vietnamese Foodies Dubai Hills Mall Phone: +971 4 880 2329. Location – Opened in August 2022. Now open, Vietnamese Foodie's fifth branch is the largest and most family-friendly. With seating for 70 people, Vietnamese ... Experience the best of Vietnamese cuisine at Foodies restaurants. An award-winning chain serves healthy and fresh meals. Try it today and taste the flavor ... Vietnamese Foodies Dubai, Dubai Hills; View reviews, menu, contact, location, and more for ... Dubai Hills Mall, Dubai ClosedOpens tomorrow at 11am. Vietnamese Foodies mission is bringing the tastiest, most healthy and authentic Vietnamese cuisine to

In [23]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0)
from langchain.chains import LLMChain,ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llm=llm,\
                                              retriever=offerdb.as_retriever(search_kwargs={"k": 5}),\
                                              qa_prompt=offer_prompt)

In [26]:
chat_history = []
query = "Who sees my credit card data?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

I'm sorry, I am unable to find the answer to your question in the document. I suggest you reach out to the human agent for more information.


In [27]:
chat_history = [(query, result["answer"])]
query = "Are there any dining options in Dubai mall?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

At Dubai Mall, you can find the following dining options: 
- Eataly: Italian restaurant 
- Vietnamese Foodies: Family restaurant 
- Roadster Diner: American restaurant 
- Eat Greek: Restaurant 
- McDonald's: Fast food restaurant
